<div style="color:white;
            display:fill;
            border-radius:15px;
            background-color:black;
            font-size:100%;
            font-family:Verdana;
            letter-spacing:1px">
    <h1 style='padding: 20px;
              color:white;
              text-align:center;'>
        Import Needed Libraries
    </h1>
    </div>
    

In [230]:
import joblib
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from plotly.offline import iplot , plot
from plotly.subplots import make_subplots
from sklearn.preprocessing import MinMaxScaler , LabelEncoder
from sklearn.model_selection import train_test_split , cross_val_score
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor , GradientBoostingRegressor , BaggingRegressor
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error , r2_score
import warnings
warnings.filterwarnings('ignore')

<div style="color:white;
            display:fill;
            border-radius:15px;
            background-color:black;
            font-size:100%;
            font-family:Verdana;
            letter-spacing:1px">
    <h1 style='padding: 20px;
              color:white;
              text-align:center;'>
        Reading Dataset And Get Info
    </h1>
    </div>

## Load Data

In [231]:
df = pd.read_csv('gld_price_data.csv')

In [232]:
# Show Sample of Data
df.sample(5)

,Date,SPX,GLD,USO,SLV,EUR/USD
2029,3/8/2017,2362.979980,115.059998,10.670000,16.32,1.057015
1523,12/5/2014,2075.370117,114.430000,24.959999,15.61,1.238666
1617,4/28/2015,2114.760010,116.330002,19.600000,15.87,1.096912
1761,12/29/2015,2078.360107,102.199997,11.260000,13.27,1.097526
258,3/16/2009,753.890015,90.800003,28.250000,12.79,1.297606


In [233]:
# Show Shape of Data
print(f"Number of Row : {df.shape[0]}\nNumber of Columns : {df.shape[1]}")

Number of Row : 2290
Number of Columns : 6


In [234]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2290 entries, 0 to 2289
Data columns (total 6 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   Date     2290 non-null   object 
 1   SPX      2290 non-null   float64
 2   GLD      2290 non-null   float64
 3   USO      2290 non-null   float64
 4   SLV      2290 non-null   float64
 5   EUR/USD  2290 non-null   float64
dtypes: float64(5), object(1)
memory usage: 107.5+ KB


In [235]:
# Check NaN Value
df.isna().sum()

Date       0
SPX        0
GLD        0
USO        0
SLV        0
EUR/USD    0
dtype: int64

In [236]:
# Describe Numiric Data
df.describe()

,SPX,GLD,USO,SLV,EUR/USD
count,2290.000000,2290.000000,2290.000000,2290.000000,2290.000000
mean,1654.315776,122.732875,31.842221,20.084997,1.283653
std,519.111540,23.283346,19.523517,7.092566,0.131547
min,676.530029,70.000000,7.960000,8.850000,1.039047
25%,1239.874969,109.725000,14.380000,15.570000,1.171313
50%,1551.434998,120.580002,33.869999,17.268500,1.303297
75%,2073.010070,132.840004,37.827501,22.882500,1.369971
max,2872.870117,184.589996,117.480003,47.259998,1.598798


In [237]:
pd.DataFrame({'Count':df.shape[0],
              'Null':df.isnull().sum(),
              'Null %':df.isnull().mean() * 100,
              'Cardinality':df.nunique()
})

,Count,Null,Null %,Cardinality
Date,2290,0,0.0,2290
SPX,2290,0,0.0,2277
GLD,2290,0,0.0,1930
USO,2290,0,0.0,1514
SLV,2290,0,0.0,1331
EUR/USD,2290,0,0.0,2066


- No Missing Value in This Data (Nulls)

<div style="color:white;
            display:fill;
            border-radius:15px;
            background-color:black;
            font-size:100%;
            font-family:Verdana;
            letter-spacing:1px">
    <h1 style='padding: 20px;
              color:white;
              text-align:center;'>
         EDA of Data and Get Insights
    </h1>
    </div>

## Convert Column 'Date' To Years to get Insights

In [238]:
# Convert and rename column in the data
df['Date'] = pd.DatetimeIndex(df['Date']).year
df.rename(columns={'Date':'Year'},inplace=True)

In [239]:
#Show Data after Changing
df.head()

,Year,SPX,GLD,USO,SLV,EUR/USD
0,2008,1447.160034,84.860001,78.470001,15.180,1.471692
1,2008,1447.160034,85.570000,78.370003,15.285,1.474491
2,2008,1411.630005,85.129997,77.309998,15.167,1.475492
3,2008,1416.180054,84.769997,75.500000,15.053,1.468299
4,2008,1390.189941,86.779999,76.059998,15.590,1.557099


In [240]:
years_in_Data = sorted(df['Year'].unique().tolist())
print(f"Number of Years in This Data '{len(years_in_Data)}' from {years_in_Data[0]} to {years_in_Data[-1]}")

Number of Years in This Data '11' from 2008 to 2018


## Average Features for a Year

In [241]:
Year_Avg_all = df.groupby('Year').mean()
colors = ["red", "gold", "snow", "seagreen", "blue"]
columns = list(Year_Avg_all.columns)
for column ,color in zip(columns,colors):
  iplot(px.line(Year_Avg_all,
                title=f'Average {column} for a Year',
                x = Year_Avg_all.index,
                y = Year_Avg_all[column],
                log_x = True,
                template = 'plotly_dark',
                markers = '*',
                line_shape = 'spline',
                color_discrete_sequence=[color]
        ))

## Correlation Matrix

In [242]:
fig = px.imshow(df.corr(), 
                text_auto=True, 
                color_continuous_scale = "tempo",  
                template='plotly_dark',
                title="Correlation Heatmap")
fig.update_layout(
    width=800, 
    height=500  
)

- GLD is highly Postivily correlated with SLV (87%)

In [243]:
# Highly Correlated
px.scatter(x = df['GLD'],
           y = df['SLV'],
           color_discrete_sequence = ['red'],
           template='plotly_dark',
           title = "Strong Positive Correlation Between GLD and SLV (87%)",
           labels = {'x':'GLD','y':'SLV'})

## Assign Feature and Target variable

In [244]:
X = df.drop(labels=['Year','GLD'] , axis=1)
y = df['GLD']

## Feature Scaling

In [245]:
scaler = MinMaxScaler()
X = scaler.fit_transform(X)

## Splitting the data into Training and Testing data

In [246]:
X_train, X_test, y_train, y_test = train_test_split(X , y , test_size = 0.25 , random_state = 44 , shuffle = True)

In [247]:
print(f'Shape of X_Train {X_train.shape}')
print(f'Shape of X_Test {X_test.shape}')
print(f'Shape of Y_Train {y_train.shape}')
print(f'Shape of Y_Test {y_test.shape}')

Shape of X_Train (1717, 4)
Shape of X_Test (573, 4)
Shape of Y_Train (1717,)
Shape of Y_Test (573,)


<div style="color:white;
            display:fill;
            border-radius:15px;
            background-color:black;
            font-size:100%;
            font-family:Verdana;
            letter-spacing:1px">
    <h1 style='padding: 20px;
              color:white;
              text-align:center;'>
         Modeling
    </h1>
    </div>

## Use Different Models Algorithm

In [248]:
def train(model,model_name):
    model.fit(X_train,y_train)
    model_train_score = model.score(X_train,y_train)
    model_test_score = model.score(X_test,y_test)
    print(f"{model_name} model score on Training data: {model_train_score * 100}%\n{model_name} model score on Testing data: {model_test_score * 100}%")

def r2(model,model_name):
    score = r2_score(y_test,model.predict(X_test))
    print(f"R2 Score for {model_name} is {score * 100}%")

## Linear Regression Model

In [249]:
lr = LinearRegression()
train(lr,'Linear Regression')
r2(lr,'Linear Regression')

Linear Regression model score on Training data: 88.12883368668508%
Linear Regression model score on Testing data: 88.16193431426868%
R2 Score for Linear Regression is 88.16193431426868%


## Random Forest Model

In [250]:
rf_model = RandomForestRegressor(n_estimators = 100 , max_depth = 10)
train(rf_model, "Random Forest")
r2(rf_model, "Random Forest")

Random Forest model score on Training data: 99.81198155207646%
Random Forest model score on Testing data: 98.7179678331627%
R2 Score for Random Forest is 98.7179678331627%


## Bagging Model

In [251]:
bag = BaggingRegressor(n_estimators= 10)
train(bag,'Bagging')
r2(bag,'Bagging')

Bagging model score on Training data: 99.74945997936206%
Bagging model score on Testing data: 98.60586620649643%
R2 Score for Bagging is 98.60586620649643%


## Gradient Boosting Model

In [252]:
GrBoost = GradientBoostingRegressor()
train(GrBoost,'Gradient Boosting')
r2(GrBoost,'Gradient Boosting')

Gradient Boosting model score on Training data: 98.86995012879898%
Gradient Boosting model score on Testing data: 97.37125873428258%
R2 Score for Gradient Boosting is 97.37125873428258%


## KNN

In [253]:
knn = KNeighborsRegressor()
train(knn,'KNN')
r2(knn,'KNN')

KNN model score on Training data: 99.36683621159924%
KNN model score on Testing data: 98.89544968223849%
R2 Score for KNN is 98.89544968223849%


## Decision Tree Regressor

In [254]:
DTR = DecisionTreeRegressor()
train(DTR,'Decision Tree')
r2(DTR,'Decision Tree')

Decision Tree model score on Training data: 100.0%
Decision Tree model score on Testing data: 97.65872670778141%
R2 Score for Decision Tree is 97.65872670778141%


## XGBoost

In [255]:
xgboost = XGBRegressor()
train(xgboost, "XGBoost")
r2(xgboost, "XGBoost")

XGBoost model score on Training data: 99.97419487748073%
XGBoost model score on Testing data: 98.57461942252405%
R2 Score for XGBoost is 98.57461942252405%
